In [73]:
import pandas as pd 
import os   
from glob import glob


In [74]:
#library to read text from pdfs
import fitz

In [75]:
#gives us the current working directory
curr_directory = os.getcwd()
print(curr_directory)

m:\AiChamps


In [76]:
#gives the directory where all pdfs are stored
profile_path = os.path.join(curr_directory + "\\profile_pdfs\\")
print(profile_path)

m:\AiChamps\profile_pdfs\


In [77]:
list_pdf_profile_paths = glob(os.path.join(profile_path,"*.{}".format('pdf')))
print(list_pdf_profile_paths[:5])

['m:\\AiChamps\\profile_pdfs\\Profile (1).pdf', 'm:\\AiChamps\\profile_pdfs\\Profile (10).pdf', 'm:\\AiChamps\\profile_pdfs\\Profile (11).pdf', 'm:\\AiChamps\\profile_pdfs\\Profile (12).pdf', 'm:\\AiChamps\\profile_pdfs\\Profile (13).pdf']


TASK 1 Complete

TASK 2 Start

In [155]:
#reading text from pdf files
def extractTextFromPDF(filepath):
    doc = fitz.open(filepath)                      # open document
    #doc object of fitz stores the document page objects
    all_text = ""
    for page in doc:
        #print(page.getText()) 
        #page.getText() returns the text contained in a single page
        all_text = all_text  + page.getText()

    return all_text
    #print(all_text)


In [156]:
test_profile_path = list_pdf_profile_paths[0] 
text_test = extractTextFromPDF(test_profile_path)
#print(text_temp)


In [157]:
#fetching text from all profiles in a list
text_all_profiles = [extractTextFromPDF(path) for path in list_pdf_profile_paths]


In [158]:
df_profiles = pd.DataFrame()

In [159]:
df_profiles['SNo'] = [i for i in range(1,51)]

In [160]:
df_profiles['Profile_Text'] = text_all_profiles

In [161]:
df_profiles.head()

,SNo,Profile_Text
0,1,\n \nContact\nbhatia.manan96@gmail.com\nwww.l...
1,2,\n \nContact\nbhatiasahil96@gmail.com\nwww.li...
2,3,\n \nContact\nakashpandey111@gmail.com\nwww.l...
3,4,\n \nContact\nNoida Sector 125\n9719207080 (M...
4,5,\n \nContact\nyashup1997@gmail.com\nwww.linke...


In [162]:
#type(df_profiles['Profile_Text'].iloc[0])

In [163]:
df_profiles.to_csv("Output\\submission_Task2.csv",index=False)

TASK 2 COMPLETE

In [164]:
from nltk import pos_tag
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import operator
import re

Rough work and explanation

In [165]:
text = text_test

Tokenizing and removing special characters

In [186]:
#\w+ captures characters,numbers and underscore and removes all special charcters as well as spaces
tokenizer = RegexpTokenizer(r'\w+')
# create English stop words list
en_stop = stopwords.words('english')
#this extendes the list of stopwords 
en_stop.extend(['www','gmail','month','page','linkedin','com','india'])
months = ['january','february','march','april','may','june','july','august','september','october','november','december']
en_stop.extend(months)
#this makes the list unique
en_stop = set(en_stop)    
text = text.lower()
#this splits the string into tokens or words
tokens = tokenizer.tokenize(text)
#print(tokens)
print(len(tokens))


417


Removing stop Words

In [187]:
# remove stop words from tokens
stopped_tokens = [j for j in tokens if not j in en_stop]
#remove one character words from tokens    
stopped_tokens = [j for j in stopped_tokens if len(j)>1]
#print(stopped_tokens)
print(len(stopped_tokens))


278


Bringing the words to its root form based on parts of speech tagging and lemmatization

In [188]:
pos_tokens = pos_tag(stopped_tokens)
print(pos_tokens[0:20])


[('contact', 'NN'), ('bhatia', 'NN'), ('manan96', 'NN'), ('manan', 'NN'), ('bhatia', 'NN'), ('top', 'JJ'), ('skills', 'NNS'), ('python', 'VBP'), ('robotic', 'JJ'), ('process', 'NN'), ('automation', 'NN'), ('rpa', 'NN'), ('google', 'NN'), ('cloud', 'NN'), ('api', 'NN'), ('certifications', 'NNS'), ('intro', 'VBP'), ('python', 'NN'), ('data', 'NNS'), ('science', 'NN')]


In [189]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        #print(wordnet.ADJ)
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV

In [190]:
tokens_final = []
wordnet_lemmatizer = WordNetLemmatizer()

for word in pos_tokens:
    try:
        lem_word = wordnet_lemmatizer.lemmatize(word[0], pos=get_wordnet_pos(word[1]))
        #print ("{0:20}{1:20}".format(word[0],lem_word))
        tokens_final.append(lem_word)
    except:
        #tokens_final.append(word[0])
        pass
        #print("\n!!Error here:",word[0],"!!\n") 
        #to handle errors where words are digits or words which are not known in English Language



In [191]:
#just to confirm removal if any new stop_words were introduced are lemmatization
tokens_final = [j for j in tokens_final if not j in en_stop]
len(tokens_final)

259

In [192]:
freq_words = {}
for word in tokens_final:
    if(word not in freq_words):
        freq_words[word] = 1
    else:
        freq_words[word]+=1 

sorted_freq_words =  sorted(freq_words.items(), key=operator.itemgetter(1),reverse=True)
print("Top 20 words frequency wise are:",sorted_freq_words[:20])

Top 20 words frequency wise are: [('process', 11), ('data', 11), ('document', 8), ('analytics', 7), ('information', 7), ('automation', 6), ('python', 5), ('extraction', 4), ('time', 4), ('bhatia', 3), ('api', 3), ('transorg', 3), ('analyst', 3), ('automate', 3), ('extract', 3), ('manan', 2), ('robotic', 2), ('rpa', 2), ('google', 2), ('cloud', 2)]


Bringing it all Together

In [206]:
def cleanAndTokenizeText(text):
    tokenizer = RegexpTokenizer(r'\w+')
    en_stop = stopwords.words('english')
    #this extendes the list of stopwords 
    en_stop.extend(['www','gmail','month','page','linkedin','com','india','year'])
    months = ['january','february','march','april','may','june','july','august','september','october','november','december']
    en_stop.extend(months)
    en_stop = set(en_stop)    
    text = text.lower()
    tokens = tokenizer.tokenize(text)
    stopped_tokens = [j for j in tokens if not j in en_stop]
    stopped_tokens = [j for j in stopped_tokens if len(j)>1]
    pos_tokens = pos_tag(stopped_tokens)
    tokens_final = []
    wordnet_lemmatizer = WordNetLemmatizer()

    for word in pos_tokens:
        try:
            lem_word = wordnet_lemmatizer.lemmatize(word[0], pos=get_wordnet_pos(word[1]))
            #print ("{0:20}{1:20}".format(word[0],lem_word))
            tokens_final.append(lem_word)
        except:
            #tokens_final.append(word[0])    #uncomment if you want to include numbers and unknown words
            pass
            #to handle errors where words are digits or words which are not known in English Language or other errors

    tokens_final = [j for j in tokens_final if not j in en_stop]
    return tokens_final


In [207]:
def getMostFrequentWords(text):
    list_words = cleanAndTokenizeText(text)
    freq_words = {}
    for word in list_words:
        if(word not in freq_words):
            freq_words[word] = 1
        else:
            freq_words[word]+=1 

    sorted_freq_words =  sorted(freq_words.items(), key=operator.itemgetter(1),reverse=True)
    most_frequent_words = [i[0] for i in sorted_freq_words]
    most_frequent_words = most_frequent_words[0:20]
    return most_frequent_words
    #print("Top 20 words freuency wise are:",sorted_freq_words[:20])


In [208]:
df_profiles['Most_Frequent_Words'] = df_profiles['Profile_Text'].apply(lambda x:getMostFrequentWords(x))

In [210]:
df_profiles.to_csv("Output\\submission_Task3.csv",index=False)